# Quantum Key Distribution
### A story of sending secrets over a quantum channel
![QKD in figures](../assets/qkd.png)
Although in modern network protocol sending secret values over the internet has become more secure than ever, there's still an issue that has not been solved and that is "how do we know if the secure data I sent from point A to B was not intercepted at a point C".

Well, the truth is in classical computing you can't really guarantee that because a bit will always be the same bit even if it gets intercepted a hundred time before getting to its destination. In other words, If I send `01101000 01101001` which is 'hi' in binary, over the network to someone on the other side of the world, there's a chance someone in the middle reads these bits and retransmits them without either the sender or the receiver ever knowing about it. This is not good because it means a secure channel could be compromised for a long time.

Enter, Quantum Key Distribution (QKD). Specifically we want to talk about BB84 QKD protocol, which promises secure transfer of secret messages and the ability to know if the message was intercepted. "But how?", you may ask. If you're into theory and wanna read the [original paper](https://web.archive.org/web/20200130165639/http://researcher.watson.ibm.com/researcher/files/us-bennetc/BB84highest.pdf) feel free. But let dive directly into code and go through how it works step by step.

In [554]:
# First let's import our libraries
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer.primitives import SamplerV2 as Sampler
from qiskit.qasm3 import dumps as getCircuitState
import re

### Step 1: Building the circuits
First, remember that through out this exercise our main goal is to send a secret key, while making sure no one intercepted our communication in the middle. Many examples use Alice and Bob as the convectional "sender" and "receiver" of the secret key. But I would like to use two distant cities of NY and SF. Imagine we have two quantum labs in these two cities connected with fibre optic cables.

In the following cell, we just setup our circuits. I have simplified this code as much as possible but feel free to experiment with your own values. Here, we have decided to send a binary secret of length 10. Because each qubit can represent 1 value, we set our circuits up with 10 qubits. 

In [555]:

num_qubits = 10
# should match num_qubits
new_york_secret_key = "0100101010"

q_reg = QuantumRegister(num_qubits, name='QBits')
c_reg = ClassicalRegister(num_qubits, name='CBits')

# Set up a circuit for the sender
new_york_qc = QuantumCircuit(q_reg,c_reg)
# A place to store the basis the sender used to send the bits
new_york_basis = []
# set up a circuit for receiver
san_francisco_qc = QuantumCircuit(q_reg, c_reg)
# A place to store the basis the receiver used to measure the bits
san_francisco_basis = []


### Step 2: Matching Circuit to Key

Our circuit need to represent our key. Meaning a qubit needs to represent 0 or 1 depending on its position. We do that by applying `x` (Pauli X) gate to the qubits that are in the same index position as a given secret bit. So if the 0th secret is a 1, we apply a `x` gate to the 0th qubit and so on.  

In [556]:
for i in range(len(new_york_secret_key)):
    if new_york_secret_key[i] == '1':
        new_york_qc.x(q_reg[i])
new_york_qc.draw()

QBits_0: ─────
          ┌───┐
 QBits_1: ┤ X ├
          └───┘
 QBits_2: ─────
               
 QBits_3: ─────
          ┌───┐
 QBits_4: ┤ X ├
          └───┘
 QBits_5: ─────
          ┌───┐
 QBits_6: ┤ X ├
          └───┘
 QBits_7: ─────
          ┌───┐
 QBits_8: ┤ X ├
          └───┘
 QBits_9: ─────
               
CBits: 10/═════

Now, we need to randomly pick a <b>basis</b> for each qubit. This is a magic word that confused me so I'll try to explain it to the best of my ability. Take a look at this photo:

![Photon polarization](../assets/photon-spin.jpg)

From the left side we can see in each example that a photon is passing through a polarization filter, that basically determines the polarization (angle) of the photon passing through. The one on the top is using a rectilinear polarizer, and the bottom one is using a diagonal polarizer. However, both of them are being "measured" through a rectilinear polarizer. So you can see, because the bottom one was measured in a different "basis" than it was polarized, there's a 50/50 chance that the measurement will be wrong. The question is: How do we simulate that on our circuit?

Here, we randomly pick New York to have basis `X` or `Z` depending on the state we set our qubit in. In this case, almost half the qubits will have an H gate and assign them basis `X` and the rest will be `Z`. You might be wondering "does it matter what basis we pick?". Well, not really. As long as New York and San Francisco agree on what basis to use, it can be anything. The important part comes later. 

<div class="alert alert-success">
<b>Important:</b> NY and SF labs already know that if a bit is 0, it is polarized either |↑> or |↗> and for 1 it's |→> or |↘>
</div>

In [557]:
for i in range(num_qubits):
    # randomly pick a basis
    choice = np.random.choice(['X','Z'])
    # remember it so we can compare with SF later
    new_york_basis.append(choice)

    # If the random basis is X, we apply a Hadamard gate to the ith qubit on NY circuit
    # This is where the "polarization" if differentiated
    if choice == 'X':
        new_york_qc.h(q_reg[i])
    

print(new_york_basis)


['X', 'Z', 'X', 'X', 'X', 'Z', 'X', 'Z', 'X', 'Z']


### Step 3: Photons, Fiber Optics and Fun!

In a real experiment, we would want to have a lab in each city, connected with fiber optic cables, and each lab equipped with photon emitters, polarization filters etc. But let's not get distracted with that overhead.

Here we simply make a function so that given the two distant circuits, we look at the state of the sender and copy it over to the receiver's circuit. Again, this is the best we can do on a $0 budget.

In [558]:
# this is a function that simulates sending actual photons from NY to SF
def SendFromNYToSF(ny_circ:QuantumCircuit, sf_circ:QuantumCircuit):
    
    # We read the state of our NY circuit 
    ny_state = getCircuitState(ny_circ).split(sep=';')[4:-1]
    ny_state = [state.lstrip() for state in ny_state]
  
    # and here we simulate sending the state of each qubit to SF using photons over fibre optic cables.
    for instruction in ny_state:
        # we simply parse the index of the current qubit
        old_qr = int(re.findall(r"\[([^\]]+)\]", instruction)[0])
        # and apply the correct gates to SF circuit depending on what we read from NY circuit
        if instruction[0] == 'x':
            sf_circ.x(q_reg[old_qr])
        elif instruction[0] == 'h':
            sf_circ.h(q_reg[old_qr])

# At this point we can imagine all photons are being sent to SF one by one
SendFromNYToSF(new_york_qc, san_francisco_qc) 

If you recall, we made a random basis list for NY. SF has no idea what order those basis where picked. Now SF need to also randomly pick the basis for each photon being sent so he can perform his measurements later. We can use the same method we used for NY here:

In [559]:
for i in range(num_qubits):
    choice = np.random.choice(['X','Z'])
    san_francisco_basis.append(choice)

    if choice == 'X':
        san_francisco_qc.h(q_reg[i])

print(san_francisco_basis)

['Z', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'X', 'X']


### Step 4: Measurement  and Final Key

After receiving all the photons and applying the right filters on them, we can finally measure them.

In [560]:
san_francisco_qc.measure_all()

print(san_francisco_qc)

          ┌───┐           ░ ┌─┐                           
 QBits_0: ┤ H ├───────────░─┤M├───────────────────────────
          ├───┤           ░ └╥┘┌─┐                        
 QBits_1: ┤ X ├───────────░──╫─┤M├────────────────────────
          ├───┤           ░  ║ └╥┘┌─┐                     
 QBits_2: ┤ H ├───────────░──╫──╫─┤M├─────────────────────
          ├───┤┌───┐      ░  ║  ║ └╥┘┌─┐                  
 QBits_3: ┤ H ├┤ H ├──────░──╫──╫──╫─┤M├──────────────────
          ├───┤├───┤┌───┐ ░  ║  ║  ║ └╥┘┌─┐               
 QBits_4: ┤ X ├┤ H ├┤ H ├─░──╫──╫──╫──╫─┤M├───────────────
          └───┘└───┘└───┘ ░  ║  ║  ║  ║ └╥┘┌─┐            
 QBits_5: ────────────────░──╫──╫──╫──╫──╫─┤M├────────────
          ┌───┐┌───┐┌───┐ ░  ║  ║  ║  ║  ║ └╥┘┌─┐         
 QBits_6: ┤ X ├┤ H ├┤ H ├─░──╫──╫──╫──╫──╫──╫─┤M├─────────
          ├───┤└───┘└───┘ ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐      
 QBits_7: ┤ H ├───────────░──╫──╫──╫──╫──╫──╫──╫─┤M├──────
          ├───┤┌───┐┌───┐ ░  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐ 

In [561]:

# Execute the SF circuit and get the counts
sampler = Sampler()

job = sampler.run([san_francisco_qc],shots=1)

result = job.result()

counts = result[0].data.meas.get_counts()

san_francisco_key = list(counts.keys())[0]
# we just simply reverse the measurement result to match the original index order
san_francisco_key = san_francisco_key[::-1]

print(san_francisco_key)


1110101010


Finally, NY and SF publicly talk to each other and share the basis they used. They will compare their measurements to the expected values, and depending on a threshold of their choosing, they can decide what "keys" to keep. 

Let's take a look at their data alongside each other:

In [562]:
print('NY basis: ',new_york_basis)
print('SF basis: ',san_francisco_basis)
print('NY Secret: ',new_york_secret_key)
print('SF Secret: ',san_francisco_key)

total_matches = 0

for i in range(num_qubits):
    if san_francisco_basis[i] == new_york_basis[i]:
        total_matches += 1

print('Basis match percentage: ', (total_matches/num_qubits) * 100)

NY basis:  ['X', 'Z', 'X', 'X', 'X', 'Z', 'X', 'Z', 'X', 'Z']
SF basis:  ['Z', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'X', 'X']
NY Secret:  0100101010
SF Secret:  1110101010
Basis match percentage:  60.0


If you run the code multiple times, you'll notice that the match percentage between the two basis is usually more than 50%. If it's less, NY and SF can decide their communication was compromised and start again.

Finally, if they agree that they have enough matching bases, they can choose the basis that matched and pick their corresponding key values as a secret.

In [563]:
shared_key = ''
for i in range(num_qubits):
    matched_basis = new_york_basis[i] == san_francisco_basis[i]
    if matched_basis:
        shared_key += san_francisco_key[i]

print('Final shared secret: ', shared_key)

Final shared secret:  101011
